In [ ]:
from google.colab import drive
isMount=!df |grep /content/drive
if len(isMount)<1: drive.mount('/content/drive')


# CNN Example From Flowers by Keras (Color Version)


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.optimizers import Adam


from os import listdir,getcwd
from os.path import isfile, join
import warnings


import numpy as np
import matplotlib.pyplot as plt


from pylab import *
from numpy import *
import cv2

import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical,get_file


IMG_W=150
IMG_H=150

In [ ]:
def getFolder(thePath,isFile=True):
    return [f for f in listdir(thePath) if isFile == isfile(join(thePath, f)) ]

def getImagesAndLabels(tPath):
    labels=getFolder(tPath,False)
    tImages,tLabels=None,None

    for label in labels:
        imgFolder=join(tPath,label)
        files= [f for f in listdir(imgFolder) if isfile(join(imgFolder, f))]
        for file in files:
            imageFile=join(imgFolder,file)
            img=plt.imread(imageFile)
            img = cv2.resize(img, (IMG_W,IMG_H))
            if tImages is None:
                tImages, tLabels =[img], [label]
            else:
                tImages.append( img )
                tLabels.append(label)
        print("finish=",label,"total=",len(tImages))
    return (tImages,tLabels,len(labels))

In [ ]:
# ![ ! -f /content/drive/MyDrive/Lec_Capture/_data/flower_photos.tgz  ]&&wget http://download.tensorflow.org/example_images/flower_photos.tgz -O /content/drive/MyDrive/Lec_Capture/_data/flower_photos.tgz 
# !tar xzf /content/drive/MyDrive/Lec_Capture/_data/flower_photos.tgz 
data_path="/content/drive/MyDrive/Lec_Capture/_data/"
url = "http://download.tensorflow.org/example_images/flower_photos.tgz"
filename = join(data_path, "flower_photos.tgz")
get_file(filename, url,extract=True)




In [ ]:
%%bash
tar xzf /content/drive/MyDrive/Lec_Capture/_data/flower_photos.tgz 
find ./flower_photos|grep jpg|head -n 10|xargs -i -t identify {} 2>>/tmp/log


In [ ]:
from sklearn.model_selection import train_test_split
basePath='./flower_photos'
imgs,labels,class_cnt=getImagesAndLabels(basePath)


In [ ]:
count=0
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range (5):
    for j in range (2):
        l=random.randint(0,len(imgs))
        ax[i,j].imshow(imgs[l])
        ax[i,j].set_title('Flower: '+ labels[l])
        ax[i,j].grid(False)
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])
        count+=1
plt.tight_layout()


In [ ]:
le=LabelEncoder()
labels=np.array(labels)

Y=le.fit_transform(labels)
Y=to_categorical(Y,class_cnt)
imgs=np.array(imgs)
imgs=imgs/255.
imgShape=imgs[0].shape

In [ ]:
train_images,test_images,train_labels,test_labels=train_test_split(imgs,Y,test_size=0.3333, random_state=99)

In [ ]:
np.random.seed(42)
random.seed(42)
tf.compat.v1.set_random_seed(42)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(96, (13, 13), activation='relu', padding='same', input_shape=imgShape))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((3, 3)))


model.add(layers.Conv2D(64, (7, 7), activation='relu', padding='same'))
model.add(layers.Dropout(0.2))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Flatten())

model.add(layers.Dense(200, activation='tanh',kernel_regularizer='l1'))
model.add(layers.Dense(class_cnt, activation='softmax'))
# model.summary()



In [ ]:


model.compile(optimizer=Adam(lr=0.001),\
              loss='categorical_crossentropy',metrics=['accuracy'])

batch_size=80
epochs=40

History = model.fit(train_images, train_labels, epochs=epochs,validation_data = (test_images,test_labels),batch_size=batch_size)

In [ ]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()


In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# getting predictions on val set.
pred=model.predict(test_images)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

print('Pred True')
for i in range(len(test_labels)):
    if(np.argmax(test_labels[i])==pred_digits[i]):
      print(np.argmax(test_labels[i]),pred_digits[i])        
      prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
print('Pred False')
for i in range(len(test_labels)):
    if(not np.argmax(test_labels[i])==pred_digits[i]):
      print(np.argmax(test_labels[i]),pred_digits[i])        
      mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:

if len(prop_class)<8:
  print("True Prediction is lower then 8")
else:

  print('--------------------------------------')
  print(' True ')
  print('--------------------------------------')
  warnings.filterwarnings('always')
  warnings.filterwarnings('ignore')

  count=0
  fig,ax=plt.subplots(4,2)
  fig.set_size_inches(15,15)
  for i in range (4):
      for j in range (2):
          img=test_images[prop_class[count]]
          ax[i,j].imshow(img)
          ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits     [prop_class[count      ]]]))+"\n"+ \
                            "Actual Flower : "   +str(le.inverse_transform([np.argmax(test_labels[prop_class[count      ]])]))    \
                          )
          ax[i,j].set_xticks([])
          ax[i,j].set_yticks([])

          plt.tight_layout()
          count+=1

In [ ]:
if len(mis_class)<8:
  print("mis_class is lower then 8")
else:
  print('--------------------------------------')
  print(' mis class')
  print('--------------------------------------')
  warnings.filterwarnings('always')
  warnings.filterwarnings('ignore')

  count=0
  fig,ax=plt.subplots(4,2)
  fig.set_size_inches(15,15)
  for i in range (4):
      for j in range (2):
          img=test_images[mis_class[count]]
          ax[i,j].imshow(img)
          ax[i,j].set_title("Predicted Flower : "+str(le.inverse_transform([pred_digits[mis_class[count]]]))+"\n"+ \
                            "Actual Flower : "   +str(le.inverse_transform([np.argmax([test_labels[mis_class[count]]])])) \
                            )

          ax[i,j].set_xticks([])
          ax[i,j].set_yticks([])
          plt.tight_layout()
          count+=1
